In [ ]:
import plotly.express as px
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import vectorbt as vbt
import warnings
import numpy as np
import plotly.io as pio
import mplfinance as mpf
pd.set_option("display.max_columns", None)

def find_relative_high(sequence, window=10):
    is_high_point = [False] * len(sequence)
    for i in range(len(sequence)):
        left_index = max(0, i - window)
        right_index = min(len(sequence), i + window + 1)
        if sequence[i] == max(sequence[left_index: right_index]):
            is_high_point[i] = True

    return is_high_point



def find_relative_low(sequence, window=10):
    is_low_point = [False] * len(sequence)
    for i in range(len(sequence)):
        left_index = max(0, i - window)
        right_index = min(len(sequence), i + window + 1)
        if sequence[i] == min(sequence[left_index: right_index]):
            is_low_point[i] = True

    return is_low_point



In [ ]:
df = pd.read_csv(r'binance_demo_LEVERUSDT_15m.csv')

In [ ]:
def High_value(High_list , length = 10 ):
    High_max = [High_list[0] for l in range(length-1)]
    for i in range(length-1,len(High_list)):
            High_max.append(max(High_list[i-length+1:i+1]))
    return High_max


High_list = list(df['Close'])
High_max = High_value(High_list,20)
df['High_max'] = High_max
df['High_max'] = df['High_max'].shift(1)

In [ ]:
df['SMA_5_H'] = df.ta.sma(closs = 'High',length =5)
df['SMA_20'] = df.ta.sma(closs = 'Close',length =20)
df['SMA_60'] = df.ta.sma(closs = 'Close',length =60)
df['SMA_60_shift'] = df['SMA_60'].shift(1)
df['Dif_H_L'] = df['High']-df['Low']
df['SMA_5_H_L'] = df.ta.sma(closs = 'Dif_H_L',length =5)
relative_high_points  = find_relative_high(list(df['SMA_5_H']),5)
relative_low_points  = find_relative_low(list(df['SMA_5_H']),5)
df['High_points'] = relative_high_points
df['Low_points'] = relative_low_points
df['High_Low_price'] = df.apply(lambda x : x['SMA_5_H'] if x['High_points'] else x['SMA_5_H']  if x['Low_points'] else None ,axis=1)
df['High_Low_mark'] = df.apply(lambda x : 'High' if x['High_points'] else 'Low'  if x['Low_points'] else None ,axis=1)
df['Date'] = pd.to_datetime(df['date'], unit='ms')
df.set_index(['Date'], inplace=True)
Volatility_line = mpf.make_addplot(df['SMA_5_H_L'], color='red', panel = 2,)
mpf.plot(df, type = 'candle', style = 'binance', title = 'Test',  addplot = [Volatility_line],volume = True)

In [ ]:
fig =go.Figure()
fig.add_traces(go.Scatter(x = df['time'] , y =df['Close'] , mode='lines+markers' , name = 'Close'))
fig.add_traces(go.Scatter(x = df['time'] , y =df['High_max'] , mode='lines+markers' , name = 'Close'))
fig.add_traces(go.Scatter(x = df['time'] , y =df['SMA_20'] , mode='lines+markers' , name = 'Close'))
fig

回測

In [ ]:
df = pd.concat([df,ta.bbands(close= df['Close'] , length=20, std=2, mamode="sma" )],axis=1)
df['BBU_20_2.0_shift'] = df['BBU_20_2.0'].shift(1)
df = df.fillna(method='bfill')
entries = (df['Close']>df['BBU_20_2.0_shift'] ) & (df['Close']>df['High_max']) & (df['Close']>df['SMA_60']) & (df['SMA_60'] > df['SMA_60_shift'])
exits = df['Close']<df['SMA_20'] 

In [ ]:
df.ta.cores=4
pf = vbt.Portfolio.from_signals(df['Close'],entries=entries ,exits=exits ,init_cash=30,fees=0.01)

print(pf.stats())

In [ ]:
# print(pf.orders.records_readable)
pf.orders.plot(width = 1000)